# API Data Wrangling

### The data:
For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), pulled from the Qunadl API. We'll analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

### The goals:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [67]:
# Store the API key (**This has been removed**)
API_KEY = 

In [68]:
# First, import the relevant modules
import requests

In [69]:
# Call the Quandl API and pull out a small sample of the data (only one day)
# to get a glimpse into the JSON structure that will be returned.

url = 'https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?start_date=2017-02-01&end_date=2017-02-01api_key=' + API_KEY
r = requests.get(url)
json_data1 = r.json()
json_data1

{'dataset_data': {'limit': None,
  'transform': None,
  'column_index': None,
  'column_names': ['Date',
   'Open',
   'High',
   'Low',
   'Close',
   'Change',
   'Traded Volume',
   'Turnover',
   'Last Price of the Day',
   'Daily Traded Units',
   'Daily Turnover'],
  'start_date': '2017-02-01',
  'end_date': '2017-02-01',
  'frequency': 'daily',
  'data': [['2017-02-01',
    34.75,
    36.0,
    34.75,
    35.94,
    None,
    85137.0,
    3038172.0,
    None,
    None,
    None]],
  'collapse': None,
  'order': None}}

### Data structure:
Inspecting the structure of the JSON object we find it is nested. The first entries include descriptive information about the data that follows.


### 1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017
+ Set start and end dates to 2017-01-01 and 2017-12-31, respectively.

In [88]:
url = 'https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?start_date=2017-01-01&end_date=2017-12-31api_key=' + API_KEY
r = requests.get(url)
json_data = r.json()

# Inspect the keys
json_data.keys()

dict_keys(['dataset_data'])

### 2. Convert the returned JSON object into a Python dictionary.

In [71]:
# The JSON file has one key, 'dataset_data'. The value is a dictionary, which we'll extract.
dataset = json_data['dataset_data']
type(dataset)

dict

In [72]:
# Identify the next level of keys.
dataset.keys()

dict_keys(['limit', 'transform', 'column_index', 'column_names', 'start_date', 'end_date', 'frequency', 'data', 'collapse', 'order'])

In [73]:
# Collect a list of the column names for reference.
col_names = dataset['column_names']
col_names

['Date',
 'Open',
 'High',
 'Low',
 'Close',
 'Change',
 'Traded Volume',
 'Turnover',
 'Last Price of the Day',
 'Daily Traded Units',
 'Daily Turnover']

In [74]:
# Extract a nested list of the daily stock data.
daily_data = dataset['data']
daily_data[0]

['2017-12-29',
 51.76,
 51.94,
 51.45,
 51.76,
 None,
 34640.0,
 1792304.0,
 None,
 None,
 None]

In [75]:
# Turn daily_data into a list of dictictionaries.
# The inner dictionary's keys are the col_names from the dataset.

day_dict = [dict(zip(col_names, day)) for day in daily_data]

# View the first two days in the list.
day_dict[:2]

[{'Date': '2017-12-29',
  'Open': 51.76,
  'High': 51.94,
  'Low': 51.45,
  'Close': 51.76,
  'Change': None,
  'Traded Volume': 34640.0,
  'Turnover': 1792304.0,
  'Last Price of the Day': None,
  'Daily Traded Units': None,
  'Daily Turnover': None},
 {'Date': '2017-12-28',
  'Open': 51.65,
  'High': 51.82,
  'Low': 51.43,
  'Close': 51.6,
  'Change': None,
  'Traded Volume': 40660.0,
  'Turnover': 2099024.0,
  'Last Price of the Day': None,
  'Daily Traded Units': None,
  'Daily Turnover': None}]

### 3. Calculate what the highest and lowest opening prices were for the stock in this period.

In [76]:
# Create a list of daily opening prices. Exclude NoneType values.
opening_prices = [day['Open'] for day in day_dict if isinstance(day['Open'], float)]
opening_prices[:3]

[51.76, 51.65, 51.45]

In [77]:
# Compute the max opening price. 
highest_opening_price = max(opening_prices)
highest_opening_price

53.11

In [78]:
# Compute the min opening price. 
lowest_opening_price = min(opening_prices)
lowest_opening_price

34.0

### 4. What was the largest change in any one day (based on High and Low price)?

In [79]:
# Create a list of price change per day by subtracting the daily High and Low prices. 
# Exclude NoneType for the High and Low values.

price_change = [day['High'] - day['Low'] for day in day_dict if isinstance(day['High'], float) and isinstance(day['Low'], float)]
price_change[:3]

[0.4899999999999949, 0.39000000000000057, 1.1300000000000026]

In [80]:
# Compute the max price change. Round to two decimal places.
largest_single_day_change = round(max(price_change), 2)
largest_single_day_change

2.81

### 5. What was the largest change between any two days (based on Closing Price)?

In [81]:
# Create a list of daily closing prices.
closing_prices = [day['Close'] for day in day_dict]
closing_prices[:5]

[51.76, 51.6, 51.82, 51.32, 51.4]

In [82]:
# Create a list of the change in closing price between one day and the next.
change_between_days = [abs(closing_prices[index] - closing_prices[index+1]) for index in range(len(closing_prices)-1)]
change_between_days[:5]

[0.1599999999999966,
 0.21999999999999886,
 0.5,
 0.0799999999999983,
 0.12999999999999545]

In [83]:
# Compute the max closing price change between days. Round to two decimal places.
largest_day2day_change = round(max(change_between_days), 2)
largest_day2day_change

2.56

### 6. What was the average daily trading volume during this year?

In [84]:
# Create a list of daily trading volume. Exclude NoneType values.
volume_traded = [day['Traded Volume'] for day in day_dict if isinstance(day['Traded Volume'], float)]
volume_traded[:5]

[34640.0, 40660.0, 57452.0, 71165.0, 120649.0]

In [85]:
# Compute the average of daily trading volume. Round to nearest whole number.
avg_volume_traded = round(sum(volume_traded)/len(volume_traded))
avg_volume_traded

89124

### 7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [86]:
def median(list):
    """Return the median value of a list of numbers.
    """
    
    # Sort the list from lowest to highest
    sorted_list = list.sort()
    
    # Find number of items in list
    n = len(list)
    
    # Find midpoint of odd length list
    if n % 2 == 1:
        median = list[n//2]
    
    # Find midpoint of even length list
    else:
        median = sum(list[n//2-1:n//2+1])/2
    
    return median

In [87]:
# Apply the median function with the list of the daily trading volume as input.
median_volume_traded = median(volume_traded)
median_volume_traded

76286.0